# Нейросети в обработке текста

В данной теме надо будет:
* Попробовать обучить нейронную сеть с применением одномерных сверток для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

* Сделать выводы

## Подключаем бибилиотеки

In [1]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

## Объявим глобальные переменные

In [2]:
MAX_WORDS = 2000
MAX_LEN = 20
NUM_CLASSES = 1

# Training
EPOCHS = 5
BATCH_SIZE = 512
PRINT_BATCH_N = 100

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

SW = set(get_stop_words('english'))
PUNCTS = set(punctuation)

tqdm.pandas()
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vlad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Попробуем обучить нейронную сеть

In [3]:
df_train = pd.read_csv("data/TextProcessing/train.csv")

### Функции, которые нам понадобятся

In [4]:
morpher = MorphAnalyzer()

# Функция для предобработки текста
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in PUNCTS)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = re.sub(r"@[\w]*", "", txt)
    txt = re.sub(r'[^\w\s]', " ", txt)
    txt = re.sub(r"[^a-zA-Z0-9]"," ", txt)
    txt = re.sub(r"[^a-zA-Z0-9]"," ", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in SW]
    return " ".join(txt)

# Функция для сборки последовательности
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

### Предобаботаем данныые

In [5]:
# предобрабатываем текст
df_train['pre_proc_tweet'] = df_train['tweet'].progress_apply(preprocess_text)

100%|██████████| 31962/31962 [00:05<00:00, 5669.91it/s]


In [6]:
# разбиваем на токены
tokens = word_tokenize(" ".join(df_train["pre_proc_tweet"]))
tokens_filtered = [word for word in tokens if word.isalnum()]

In [7]:
# создаем словарь
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]  # вычитание 1 для padding
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [13]:
%%time
x_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in df_train["pre_proc_tweet"]])

CPU times: total: 2.67 s
Wall time: 2.72 s


### Опишем нейронную сеть с dataset

In [14]:
class TextProcessingNet(nn.Module):
    """
    Описанная нейронная сеть для данных из твита
    """
    def __init__(self, vocab_size=MAX_WORDS, embedding_dim=128, out_channel=128, num_classes = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.block_1 = nn.Sequential(
            nn.Conv1d(embedding_dim, out_channel, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv1d(embedding_dim, out_channel, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.block_3 = nn.Sequential(
            nn.Linear(out_channel, out_channel // 2),
            nn.ReLU(),
            nn.Linear(out_channel // 2, num_classes)
        )

        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        output = self.embedding(x)
        output = output.permute(0, 2, 1)
        output = self.block_1(output)

        output = self.block_2(output)
        output = torch.max(output, axis=2).values
        output = self.block_3(output)
        output = self.sigmoid(output)
        return output

In [15]:
class TextProcessingDataset(Dataset):
    """
    Датасет для нейронной сети
    """
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.data)

### Обучим нейронную сеть

In [16]:
# Загружаем данные в dataset
train_dataset = TextProcessingDataset(x_train, df_train['label'].values)
# Разбиваем наши данные на тренеровочные и тестовые с параметрамми для трейна = 67% и теста = 33%
train_dataset, test_dataset = train_test_split(train_dataset, train_size=0.67, test_size=0.33)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [17]:
# создаем нашу модель
model = TextProcessingNet(vocab_size=MAX_WORDS)

In [18]:
# Оптимизатор и функция потерь
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [19]:
model = model.to(DEVICE)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in range(EPOCHS):
    running_items, running_right = 0.0, 0.0
    for i, train_data in enumerate(train_loader, 0):
        inputs, labels = train_data[0].to(DEVICE), train_data[1].to(DEVICE)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

        # выводим статистику о процессе обучения
        if i % 150 == 0:    # печатаем каждые 150 batches
            model.eval()

            print(f'Epoch [{epoch + 1}/{EPOCHS}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {loss:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            train_loss_history.append(loss)

            # выводим статистику на тестовых данных
            test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
            for j, test_data in enumerate(test_loader):
                test_labels = test_data[1]
                test_outputs = model(test_data[0])

                # подсчет ошибки на тесте
                test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                # подсчет метрики на тесте
                test_running_total += len(test_data[1])
                pred_test_labels = torch.squeeze((test_outputs > th).int())
                test_running_right += (test_labels == pred_test_labels).sum()

            test_loss_history.append(test_loss.item())
            print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')

        model.train()

print('Training is finished!')

Epoch [1/5]. Step [1/42]. Loss: 0.811. Acc: 0.064. Test loss: 0.826. Test acc: 0.068
Epoch [2/5]. Step [1/42]. Loss: 0.342. Acc: 0.934. Test loss: 0.246. Test acc: 0.932
Epoch [3/5]. Step [1/42]. Loss: 0.263. Acc: 0.928. Test loss: 0.086. Test acc: 0.932
Epoch [4/5]. Step [1/42]. Loss: 0.312. Acc: 0.908. Test loss: 0.073. Test acc: 0.932
Epoch [5/5]. Step [1/42]. Loss: 0.241. Acc: 0.936. Test loss: 0.070. Test acc: 0.932
Training is finished!


## Вывод

Было проведено 3 тестрирования (результаты представлены на результате 5-ой эпохи):

1) С оптимизатором **SGD**, в результате чего у нейронной сети:
 * *test*: lost - 0.241, acc - 0.936
 * *train*: lost - 0.070, acc - 0.932

2) С оптимизатором **Adam** и **минимум предобаботки теста**:
 * *test*: lost - 0.047, acc - 0.982
 * *train*: lost - 0.033, acc - 0.944

3) С оптимизатором **Adam** и **дополнительной предобаботки теста**:
 * *test*: lost - 0.045, acc - 0.984
 * *train*: lost - 0.000, acc - 0.950

**Результат**: обучать нейронную сеть лучше используя Adam, так же лучше как можно лучше заниматься предобработкой теста, убирая лишние данные, чтобы не сильно засорять обучение нейронной сети.